# AI/ML i GenAI Integrations

**Cel szkoleniowy:** Zrozumienie mozliwosci AI/ML w Databricks: MLflow, Feature Store, Vector Search i GenAI

**Zakres tematyczny:**
- MLflow: Tracking, Registry, Model Serving
- Feature Store: centralne repozytorium cech
- Vector Search: baza wektorowa dla RAG
- GenAI: integracja z LLM (DBRX, Llama)


## Setup i konfiguracja

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 3 - Integracje i Governance
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
 - Klaster ML (Databricks Runtime ML 13.0+)
 - Dostęp do Unity Catalog (dla Feature Store i Vector Search)
 - (Opcjonalnie) Skonfigurowany Vector Search Endpoint
 - (Opcjonalnie) Dostęp do Model Serving (Foundation Models)

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie jak Databricks wspiera pełny cykl życia AI/ML.

**Podstawowe pojęcia:**
- **MLflow**: Platforma open-source do zarządzania cyklem życia ML (eksperymenty, modele, wdrożenia).
- **Feature Engineering (Feature Store)**: Scentralizowane repozytorium cech w Unity Catalog, zapewniające spójność między treningiem a inferencją.
- **Vector Search**: Baza danych wektorowych zintegrowana z Delta Lake, kluczowa dla RAG (Retrieval Augmented Generation).
- **Model Serving**: Wdrażanie modeli jako endpointy REST API (w tym Foundation Models jak DBRX, Llama).

## Izolacja per użytkownik

In [0]:
%run ../00_setup

## Konfiguracja środowiska

In [0]:
import mlflow
import pandas as pd
from pyspark.sql import functions as F
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Feature Engineering Client (Unity Catalog - nowy sposób)
from databricks.feature_engineering import FeatureEngineeringClient

# Ustawienie katalogu i schematu
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {GOLD_SCHEMA}")

display(spark.createDataFrame([
    ("Katalog", CATALOG),
    ("Schemat", GOLD_SCHEMA)
], ["Parametr", "Wartość"]))

## Sekcja 1: MLflow - Tracking & Registry

Zbudujemy prosty model przewidujący wartość zamówienia i zarejestrujemy go w MLflow.

In [0]:
# 1. Konfiguracja eksperymentu
username = spark.sql("SELECT current_user()").collect()[0][0]
experiment_path = f"/Users/{username}/KION_ML_Experiment"

mlflow.set_experiment(experiment_path)

display(spark.createDataFrame([("Eksperyment MLflow", experiment_path)], ["Parametr", "Wartość"]))

In [0]:
%python
# 2. Przygotowanie danych
df_sales = spark.sql(
    f"""
    SELECT 
        net_amount as label,
        quantity,
        month(order_ts) as month,
        year(order_ts) as year
    FROM {CATALOG}.{GOLD_SCHEMA}.fact_sales
    WHERE net_amount IS NOT NULL
    """
).toPandas()

X = df_sales.drop("label", axis=1)
y = df_sales["label"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

display(
    spark.createDataFrame(
        [
            ("Train shape", f"{X_train.shape[0]} rows, {X_train.shape[1]} features"),
            ("Test shape", f"{X_test.shape[0]} rows, {X_test.shape[1]} features")
        ],
        ["Dataset", "Rozmiar"]
    )
)

In [0]:
# 3. Trening i logowanie
with mlflow.start_run(run_name="RandomForest_v1") as run:
    # Parametry
    n_estimators = 50
    mlflow.log_param("n_estimators", n_estimators)
    
    # Model
    model = RandomForestRegressor(n_estimators=n_estimators)
    model.fit(X_train, y_train)
    
    # Metryki
    mse = mean_squared_error(y_test, model.predict(X_test))
    mlflow.log_metric("mse", mse)
    
    # Logowanie modelu
    mlflow.sklearn.log_model(model, "model")
    
    run_id = run.info.run_id

display(spark.createDataFrame([
    ("Run ID", run_id),
    ("MSE", f"{mse:.2f}"),
    ("Model", "RandomForestRegressor"),
    ("n_estimators", str(n_estimators))
], ["Parametr", "Wartość"]))

## Sekcja 2: Feature Store

Feature Store pozwala na definiowanie cech raz i używanie ich wszędzie.

In [0]:
from databricks.feature_store import FeatureStoreClient

In [0]:
# Definicja tabeli cech (Feature Table)
fs = FeatureStoreClient()
feature_table_name = f"{CATALOG}.{GOLD_SCHEMA}.customer_features_{username.split('@')[0].replace('.', '_')}"

# Logika obliczania cech
features_df = spark.sql(
    f"""
    SELECT 
        customer_id,
        count(*) as order_count,
        sum(net_amount) as total_spend,
        avg(net_amount) as avg_order_value
    FROM {CATALOG}.{GOLD_SCHEMA}.fact_sales
    GROUP BY customer_id
    """
)

# Zapis do Feature Store
# Uwaga: create_table wymaga unikalnej nazwy i Primary Key

fs.create_table(
    name=feature_table_name,
    primary_keys=["customer_id"],
    df=features_df,
    description="Podstawowe cechy klienta: liczba zamówień, wydatki."
)
status_msg = "Utworzona"


display(spark.createDataFrame([
    ("Feature Table", feature_table_name),
    ("Status", status_msg),
    ("Primary Key", "customer_id"),
    ("Cechy", "order_count, total_spend, avg_order_value")
], ["Parametr", "Wartość"]))

## Sekcja 3: Vector Search (RAG Foundation)

Vector Search to klucz do budowania systemów RAG. Pozwala wyszukiwać dokumenty podobne semantycznie do zapytania użytkownika.

**Wymagania:**
1. **Vector Search Endpoint** (musi być utworzony w Compute -> Vector Search).
2. **Tabela źródłowa** z włączonym Change Data Feed.

In [0]:
# 1. Przygotowanie danych tekstowych (Symulacja bazy wiedzy)
# Tworzymy tabelę z opisami produktów (w realnym scenariuszu to byłyby PDFy, dokumentacja itp.)

docs_df = spark.createDataFrame([
    (1, "Wózek widłowy elektryczny, udźwig 2 tony, do pracy wewnątrz magazynu."),
    (2, "Wózek spalinowy Diesel, udźwig 5 ton, do pracy na zewnątrz, trudny teren."),
    (3, "Ręczny wózek paletowy, udźwig 500kg, do lekkich prac."),
    (4, "System automatyzacji magazynu, regały wysokiego składowania.")
], ["id", "text"])

source_table = f"{CATALOG}.{GOLD_SCHEMA}.product_docs_rag"
docs_df.write.format("delta").mode("overwrite").option("delta.enableChangeDataFeed", "true").saveAsTable(source_table)

display(spark.createDataFrame([
    ("Tabela źródłowa", source_table),
    ("Liczba dokumentów", "4"),
    ("Change Data Feed", "Włączony")
], ["Parametr", "Wartość"]))

In [0]:

%skip 
%pip install databricks-vectorsearch
dbutils.library.restartPython()

In [0]:

# 2. Konfiguracja Vector Search (Kod właściwy)
# Uwaga: To wymaga aktywnego Endpointu Vector Search. 
# Jeśli go nie ma, kod rzuci błąd, ale pokazuje poprawną ścieżkę implementacji.

from databricks.vector_search.client import VectorSearchClient

# Nazwa endpointu (musi istnieć w UI)
VECTOR_SEARCH_ENDPOINT_NAME = "product_docs_index"
index_name = f"{CATALOG}.{GOLD_SCHEMA}.product_docs_idx"

status_info = []

try:
    vsc = VectorSearchClient()
    # Tworzenie indeksu (Delta Sync Index)
    vsc.create_delta_sync_index(
        endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
        source_table_name=source_table,
        index_name=index_name,
        pipeline_type="TRIGGERED",
        primary_key="id",
        embedding_source_column="text",
        embedding_model_endpoint_name="databricks-gte-large-en"
    )
    status_info.append(("Status", "Indeks utworzony"))
    status_info.append(("Endpoint", VECTOR_SEARCH_ENDPOINT_NAME))
    status_info.append(("Index Name", index_name))
except Exception as e:
    status_info.append(("Status", f"Błąd: {e}"))

display(spark.createDataFrame(status_info, ["Parametr", "Wartość"]))

In [0]:

# 3. Symulacja wyszukiwania (gdyby Vector Search działał)

query = "Szukam wózka do ciężkich prac na dworze"
display(spark.sql(f"""
SELECT
    *
FROM
    VECTOR_SEARCH(
        index => '{index_name}',
        query_text => '{query}',
        num_results => 3
    )
"""))

In [0]:

# --- SYMULACJA LOKALNA (Dla celów edukacyjnych, gdy brak infrastruktury) ---
display(spark.createDataFrame([
    ("Query", "Szukam wózka do ciężkich prac na dworze"),
    ("", ""),
    ("Wynik - ID", "2"),
    ("Wynik - Text", "Wózek spalinowy Diesel, udźwig 5 ton, do pracy na zewnątrz, trudny teren."),
    ("Wynik - Score", "0.89")
], ["Parametr", "Wartość"]))

## Sekcja 4: GenAI & Model Serving

Integracja z modelami LLM (Large Language Models) poprzez Databricks Model Serving.

In [0]:
# Wywołanie modelu LLM (np. DBRX, Llama 3)
import mlflow.deployments

# Lista dostępnych endpointów (może wymagać uprawnień)
# client = mlflow.deployments.get_deploy_client("databricks")
# endpoints = client.list_endpoints()

def query_llm(prompt):
    try:
        client = mlflow.deployments.get_deploy_client("databricks")
        
        # Używamy modelu 'databricks-claude-sonnet-4-5' lub 'databricks-meta-llama-3-70b-instruct'
        # Te modele są często dostępne jako "Pay-per-token"
        response = client.predict(
            endpoint="databricks-claude-sonnet-4-5",
            inputs={
                "messages": [
                    {"role": "system", "content": "Jesteś ekspertem od wózków widłowych."},
                    {"role": "user", "content": prompt}
                ],
                "max_tokens": 100
            }
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"Nie udało się połączyć z modelem: {e}"

# Test
prompt = "Jakie są zalety wózków elektrycznych?"
odpowiedz = query_llm(prompt)

display(spark.createDataFrame([
    ("Pytanie", prompt),
    ("Odpowiedź LLM", odpowiedz)
], ["Parametr", "Wartość"]))

## Podsumowanie

1. Zbudowaliśmy pipeline treningowy w **MLflow**.
2. Utworzyliśmy tabelę cech w **Feature Store**.
3. Pokazaliśmy jak skonfigurować **Vector Search** dla RAG.
4. Skorzystaliśmy z **GenAI** do generowania odpowiedzi.

## Czyszczenie zasobów

In [0]:
# spark.sql(f"DROP TABLE IF EXISTS {feature_table_name}")
# spark.sql(f"DROP TABLE IF EXISTS {source_table}")
display(spark.createDataFrame([("Status", "Zasoby zachowane do dalszych ćwiczeń")], ["Info", "Wartość"]))